In [1]:
from torch.utils.data import DataLoader
from streaming import StreamingDataset
import time
from tqdm import tqdm   
import torch

/root/miniconda3/envs/videos/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import streaming
streaming.base.util.clean_stale_shared_memory()

# Option 2

In [2]:
# Remote directory (S3 or local filesystem) where dataset is stored
remote_dir = '/root/projects/rl-nlp/videos/data_downsampled/mds/00000'

# Local directory where dataset is cached during operation
local_dir = '/root/projects/rl-nlp/videos/cache/split'
dataset = StreamingDataset(local=local_dir, remote=remote_dir, split=None, shuffle=False)

# Create PyTorch DataLoader
dataloader_split = DataLoader(dataset)

In [3]:
start = time.time()
for i, data in enumerate(tqdm(dataloader_split)):
    break
end = time.time() - start
print(end)

  0%|          | 0/247 [00:00<?, ?it/s]/root/miniconda3/envs/videos/lib/python3.11/site-packages/streaming/base/dataset.py:722: RuntimeWarning: divide by zero encountered in scalar floor_divide
  shard_shuffle_units = [shard_samples] * (shard_choose // shard_samples)
/root/miniconda3/envs/videos/lib/python3.11/site-packages/streaming/base/dataset.py:723: RuntimeWarning: divide by zero encountered in scalar remainder
  remainder = shard_choose % shard_samples
/root/miniconda3/envs/videos/lib/python3.11/site-packages/streaming/base/dataset.py:730: RuntimeWarning: divide by zero encountered in scalar floor_divide
  num_full_repeats = shard_choose // shard_samples
/root/miniconda3/envs/videos/lib/python3.11/site-packages/streaming/base/dataset.py:738: RuntimeWarning: divide by zero encountered in scalar remainder
  shortfall = shard_choose % shard_samples
/root/miniconda3/envs/videos/lib/python3.11/site-packages/torch/utils/data/_utils/collate.py:171: UserWarning: The given NumPy array is 

4.483183860778809


In [6]:
frame = data['frame']
subtitle = data['subtitle']
batch_size = frame.shape[0]
num_frames = frame.shape[1]
num_chunks = frame.shape[2]
frame.shape

torch.Size([1, 333, 1, 360, 640, 3])

In [7]:
subtitle.shape

torch.Size([1, 333, 1, 40])

In [8]:
subtitle[0, 0, 0, :].numpy().tobytes().decode('utf-8')

'every family has their good seeds and\x00\x00\x00'

# Option 1

In [3]:
# Remote directory (S3 or local filesystem) where dataset is stored
remote_dir = '/root/projects/rl-nlp/videos/data_downsampled/mds-not-split/00000'

# Local directory where dataset is cached during operation
local_dir = '/root/projects/rl-nlp/videos/cache/not-split'
dataset = StreamingDataset(local=local_dir, remote=remote_dir, split=None, shuffle=False)

# Create PyTorch DataLoader
dataloader_not_split = DataLoader(dataset)

In [6]:
start = time.time()
chunked_frames = []
chunked_subtitle = []
chunk_unit = 1
for i, data in enumerate(tqdm(dataloader_not_split)):
    # NOTE: If you wanna do just a equal splitting, just do torch.chunk(data['frame'], num_chunked_frames, dim=1)
    chunk_start_pos = 0
    for j in range(data['frame'].shape[1]):
        if (j > 0) and (subtitle != data['subtitle'][0, j, :].numpy().tobytes().decode('utf-8')):
            num_chunks = (j - chunk_start_pos) // chunk_unit
            chunked_frames += torch.chunk(data['frame'][0, chunk_start_pos:j, :, :, :], num_chunks, dim=1)
            chunked_subtitle += torch.chunk(data['subtitle'][0, chunk_start_pos:j, :], num_chunks, dim=1)
            chunk_start_pos = j
        subtitle = data['subtitle'][0, j, :].numpy().tobytes().decode('utf-8')
    break
end = time.time() - start
print(end)

  0%|          | 0/247 [00:05<?, ?it/s]

6.124604940414429
